### **HuggingFace Login**

In [10]:
from huggingface_hub import login
login()

### **Import Libairies**

In [1]:
from IPython.display import display

import pandas as pd
import numpy as np

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import Callback

import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, f1_score
from datasets import Dataset


### **Read Datasets**

In [2]:
'''
df_train = pd.read_csv('/media/yanncauchepin/ExternalDisk/Datasets/ComputerVisionImages/digit_recognizer/train.csv')
df_test = pd.read_csv('/media/yanncauchepin/ExternalDisk/Datasets/ComputerVisionImages/digit_recognizer/test.csv')
'''
df_train = pd.read_csv('C:/Users/cauchepy/Datasets/ComputerVisionImages/kaggle_digitrecognizer/train.csv')
df_test = pd.read_csv('C:/Users/cauchepy/Datasets/ComputerVisionImages/kaggle_digitrecognizer/test.csv')

### **Short Analysis**

In [3]:
distinct_labels = df_train['label'].value_counts()
display(pd.DataFrame(distinct_labels))
classes = len(df_train["label"].unique())

,count
label,
1,4684
7,4401
3,4351
9,4188
2,4177
6,4137
0,4132
4,4072
8,4063


### **Preprocess Datasets**

In [4]:
y_train = np.array(df_train['label'])
X_train = np.array(df_train.drop('label', axis=1))
X_test = np.array(df_test)

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

In [6]:
X_train_norm = X_train / 255.0
X_valid_norm = X_valid / 255.0
X_test_norm = X_test / 255.0

### **Model Assessment**

In [7]:
def evaluate_classifier(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    metrics_df = pd.DataFrame({
        'Metric': ['F1 Score', 'Precision', 'Recall'],
        'Value': [f1, precision, recall]
    })
    
    cm_df = pd.DataFrame(cm, columns=df_train["label"].unique(), index=df_train["label"].unique())
    
    return metrics_df, cm_df

### **Transformers VIT**

In [11]:
from transformers import ViTForImageClassification, AdamW
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

# Initialize ViT model
vit_classifier = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k', 
    num_labels=10
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_classifier.to(device)

# Training setup
optimizer = AdamW(vit_classifier.parameters(), lr=1e-5)
epochs = 3

# Training loop
for epoch in range(epochs):
    vit_classifier.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = vit_classifier(inputs, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss:.4f}")

# Evaluation on validation set
vit_classifier.eval()
all_vit_classifier_preds, all_vit_classifier_labels = [], []

with torch.no_grad():
    for batch in valid_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = vit_classifier(inputs)
        preds = torch.argmax(outputs.logits, axis=1)
        
        all_vit_classifier_preds.extend(preds.cpu().numpy())
        all_vit_classifier_labels.extend(labels.cpu().numpy())

# Calculate accuracy and classification report
accuracy = accuracy_score(all_vit_classifier_labels, all_vit_classifier_preds)
print(f"Validation Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(all_vit_classifier_labels, all_vit_classifier_preds))

# Additional assessment (if function `evaluate_classifier` is defined)
vit_classifier_assessment = evaluate_classifier(all_vit_classifier_labels, all_vit_classifier_preds)
print(vit_classifier_assessment)

vit_classifier.push_to_hub("yanncauchepin/kaggle_digitrecognizer_vit_model")

2024-11-02 23:02:56.570102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-02 23:02:56.739895: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-02 23:02:56.783626: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 23:02:57.069157: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-02 23:02:58.956709: W tensorflow/compiler/tf2

Epoch 1/3, Training Loss: 0.6977


100%|██████████| 11200/11200 [1:34:05<00:00,  1.98it/s]


Epoch 2/3, Training Loss: 0.2149


100%|██████████| 11200/11200 [1:33:56<00:00,  1.99it/s]


Epoch 3/3, Training Loss: 0.1533
Validation Accuracy: 0.9501
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.97      0.98       827
           1       0.98      0.99      0.99       937
           2       0.98      0.91      0.94       835
           3       0.89      0.92      0.90       870
           4       0.99      0.94      0.97       814
           5       0.84      0.98      0.91       759
           6       0.99      0.97      0.98       827
           7       0.97      0.96      0.97       880
           8       0.95      0.91      0.93       813
           9       0.94      0.94      0.94       838

    accuracy                           0.95      8400
   macro avg       0.95      0.95      0.95      8400
weighted avg       0.95      0.95      0.95      8400

(      Metric     Value
0   F1 Score  0.950554
1  Precision  0.952667
2     Recall  0.950119,      1    0    4    7    3    5    8    9    2    6
1  799    

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yanncauchepin/kaggle_digitrecognizer_vit_model/commit/00301fd101de7d31c7859b0db55ee00300b73253', commit_message='Upload ViTForImageClassification', commit_description='', oid='00301fd101de7d31c7859b0db55ee00300b73253', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yanncauchepin/kaggle_digitrecognizer_vit_model', endpoint='https://huggingface.co', repo_type='model', repo_id='yanncauchepin/kaggle_digitrecognizer_vit_model'), pr_revision=None, pr_num=None)

In [ ]:
def save_data_to_disk(data, labels, transform, path, prefix="train"):
    for i, (img, label) in enumerate(zip(data, labels)):
        img_tensor = transform(img)  # Apply transformations
        
        # Save only the image tensor if the label is None
        if label is None:
            torch.save(img_tensor, path / f"{prefix}_{i}.pt")
        else:
            torch.save((img_tensor, torch.tensor(label)), path / f"{prefix}_{i}.pt")

save_data_to_disk(X_test, [None] * len(X_test), transform, external_path, prefix="test")  # Label is None for test

class DiskDataset(Dataset):
    def __init__(self, path, prefix="train", include_labels=True):
        self.path = path
        self.files = list(path.glob(f"{prefix}_*.pt"))
        self.include_labels = include_labels

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        data = torch.load(self.files[idx])
        
        # Return data accordingly (without labels for test data)
        if self.include_labels:
            img_tensor, label = data
            return img_tensor, label
        else:
            return data 

# Create a custom Dataset and DataLoader for the test set
test_dataset = DiskDataset(external_path, prefix="test", include_labels=False)
test_loader = DataLoader(test_dataset, batch_size=3)

vit_classifier.eval()
test_preds = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        inputs = batch.to(device)  
        outputs = vit_classifier(inputs)
        preds = torch.argmax(outputs.logits, axis=1)
        test_preds.extend(preds.cpu().numpy())

vit_classifier_submission_df = pd.DataFrame({
    "ImageId": range(1, len(test_preds) + 1),
    "Label": test_preds
})
from datasets import Dataset
Dataset.from_pandas(vit_classifier_submission_df).push_to_hub("yanncauchepin/kaggle_digitrecognizer_vit_submission_df")


  0%|          | 0/9334 [00:00<?, ?it/s]/tmp/ipykernel_25585/3633612418.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(self.files[idx])
100%|████████

AttributeError: type object 'Dataset' has no attribute 'from_pandas'